In [1]:
#First Block - LOADING - Welcome to Aurora

PROJECT_ID = "qwiklabs-gcp-03-482902ef4b4f"

from google.cloud import bigquery
bq_client = bigquery.Client(project=PROJECT_ID)

DATASET_ID = "aurora_bay"
RAW_TABLE_ID = f"{PROJECT_ID}.{DATASET_ID}.faqs_raw"

# Create dataset if it doesn't exist
dataset_ref = bigquery.Dataset(f"{PROJECT_ID}.{DATASET_ID}")
dataset_ref.location = "US"
bq_client.create_dataset(dataset_ref, exists_ok=True)
print(f"Dataset {DATASET_ID} ready.")

Dataset aurora_bay ready.


In [2]:
#Second Block - LOAD FAQs - Does Aurora Have...?

job_config = bigquery.LoadJobConfig(
    source_format=bigquery.SourceFormat.CSV,
    skip_leading_rows=1,  # Skip the header row
    autodetect=True,      # Automatically detect schema
    write_disposition="WRITE_TRUNCATE"  # Overwrite table instead of appending
)

with open("aurora-bay-faqs.csv", "rb") as source_file:
    load_job = bq_client.load_table_from_file(
        source_file,
        RAW_TABLE_ID,
        job_config=job_config,
    )

load_job.result()  # Waits for the job to complete

table = bq_client.get_table(RAW_TABLE_ID)
print(f"Loaded {table.num_rows} rows and {len(table.schema)} columns to {RAW_TABLE_ID}")


Loaded 50 rows and 2 columns to qwiklabs-gcp-03-482902ef4b4f.aurora_bay.faqs_raw


In [3]:
#Third Block - EMBEDDINGGS - The History of Aurora

import bigframes.pandas as bpd
from typing import List


RAW_TABLE_ID = "qwiklabs-gcp-03-482902ef4b4f.aurora_bay.faqs_raw"
df_faqs = bpd.read_gbq(RAW_TABLE_ID)
df_faqs.peek()

,string_field_0,string_field_1
10,What local outdoor adventure companies operate...,North Star Excursions and Bay Explorers offer ...
31,Is there a local community center for events a...,Yes. The Aurora Bay Community Center on Seavie...
37,What cultural events are unique to Aurora Bay?,"Besides the Aurora Lights Winter Festival, the..."
1,When was Aurora Bay founded?,Aurora Bay was founded in 1901 by a group of f...
47,When is the annual Salmon Derby?,The annual Salmon Derby takes place in early J...


In [10]:
#Fourth Block - SET SCHEMA - Building som Tables

bq_client.query("""
DROP TABLE IF EXISTS `qwiklabs-gcp-03-482902ef4b4f.aurora_bay.faqs_with_embeddings`
""").result()

bq_client.query("""
CREATE TABLE `qwiklabs-gcp-03-482902ef4b4f.aurora_bay.faqs_with_embeddings` (
  id INT64,
  question STRING,
  answer STRING,
  embedding ARRAY<FLOAT64>
)
""").result()

print("Embeddings table recreated with correct schema.")


Embeddings table recreated with correct schema.


In [5]:
#Fifth Block - ORDERING HELPER - Keeping it Real

from bigframes.ml.llm import TextEmbeddingGenerator
QUERY_EMBEDDING_MODEL = "text-embedding-005"


In [6]:
#SIXRH BLOCK - FAQs - WHho, What, Where...

def get_similar_faqs(query: str, top_k: int = 3) -> List[dict]:
    """
    Performs a vector search using ML.DISTANCE() on ARRAY<FLOAT64>.
    Compatible with Qwiklabs BigQuery environments.
    """

    # Generate embedding for the query
    embedding_model_for_query = TextEmbeddingGenerator(model_name=QUERY_EMBEDDING_MODEL)
    df_query_text = bpd.DataFrame({"content": [query]})
    query_embedding_bf = embedding_model_for_query.predict(df_query_text)
    query_embedding_list = query_embedding_bf['ml_generate_embedding_result'].to_pandas().iloc[0]

    # Convert embedding to SQL array literal
    query_embedding_sql_array = str(list(query_embedding_list))

    # FINAL SQL — matches your table schema EXACTLY
    vector_search_query = f"""
    SELECT
        question,
        answer,
        ML.DISTANCE(
            embedding,
            ARRAY<FLOAT64>{query_embedding_sql_array},
            'COSINE'
        ) AS similarity_score
    FROM `{OUTPUT_TABLE_ID}`
    ORDER BY similarity_score
    LIMIT {top_k}
    """

    query_job = bq_client.query(vector_search_query)
    results = query_job.result()

    similar_faqs = []
    for row in results:
        similar_faqs.append({
            "question": row.question,
            "answer": row.answer,
            "similarity_score": float(row.similarity_score)
        })

    return similar_faqs


In [7]:
#Seventh Block - LOADING IT ALL - Ventor Search

# Step 2.5 — Load embedding model
from vertexai.language_models import TextEmbeddingModel
model = TextEmbeddingModel.from_pretrained("text-embedding-005")

OUTPUT_TABLE_ID = "qwiklabs-gcp-03-482902ef4b4f.aurora_bay.faqs_with_embeddings"
print("OUTPUT_TABLE_ID =", OUTPUT_TABLE_ID)

# Step 3: Insert embeddings into BigQuery
rows_to_insert = []

for idx, row in df_faqs.iterrows():
    q = str(row["string_field_0"])
    a = str(row["string_field_1"])

    text = q + "\n" + a

    emb = model.get_embeddings([text])[0].values
    emb = [float(x) for x in emb]

    rows_to_insert.append({
        "id": int(idx),
        "question": q,
        "answer": a,
        "embedding": emb
    })

errors = bq_client.insert_rows_json(OUTPUT_TABLE_ID, rows_to_insert)

if errors:
    print("Errors inserting rows:", errors)
else:
    print("Embeddings successfully inserted into BigQuery.")




OUTPUT_TABLE_ID = qwiklabs-gcp-03-482902ef4b4f.aurora_bay.faqs_with_embeddings


/usr/local/lib/python3.12/dist-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


Embeddings successfully inserted into BigQuery.


In [8]:
#Eighth Block - BRING ON THE CHATBOT

def chatbot_interaction():
    """
    Runs the main loop for the chatbot, handling user input and displaying search results.
    """
    print("Welcome to the Aurora Bay FAQ Chatbot! Type 'exit' or 'quit' to end the conversation.")

    while True:
        user_query = input("\nYour question: ")

        if user_query.lower() in ('exit', 'quit'):
            print("Thank you for using the Aurora Bay FAQ Chatbot. Goodbye!")
            break

        print(f"Searching for answers to: '{user_query}'...")

        # Call the get_similar_faqs function to find relevant FAQs
        similar_faqs = get_similar_faqs(user_query, top_k=3)

        if similar_faqs:
            print("\nHere are the most relevant FAQs I found:")
            for i, faq in enumerate(similar_faqs):
                print(f"---\nResult {i+1} (Similarity: {faq['similarity_score']:.4f}):")
                print(f"Question: {faq['question']}")
                print(f"Answer: {faq['answer']}")
            print("\n---")
        else:
            print("Sorry, I couldn't find any relevant FAQs for your question. Please try rephrasing.")

print("Defined `chatbot_interaction` function.")

Defined `chatbot_interaction` function.


In [9]:
#Ninth Block - THE MIGHTY CHATBOT TEST

print("Demonstrating chatbot interaction with a sample question:")
chatbot_interaction()

Demonstrating chatbot interaction with a sample question:
Welcome to the Aurora Bay FAQ Chatbot! Type 'exit' or 'quit' to end the conversation.

Your question: How do I adopt a pet from the local shelter?
Searching for answers to: 'How do I adopt a pet from the local shelter?'...


/usr/local/lib/python3.12/dist-packages/bigframes/dtypes.py:987: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)
/usr/local/lib/python3.12/dist-packages/bigframes/dtypes.py:987: JSONDtypeWarning: JSON columns will be represented as pandas.ArrowDtype(pyarrow.json_())
instead of using `db_dtypes` in the future when available in pandas
(https://github.com/pandas-dev/pandas/issues/60958) and pyarrow.
  warnings.warn(msg, bigframes.exceptions.JSONDtypeWarning)



Here are the most relevant FAQs I found:
---
Result 1 (Similarity: 0.1995):
Question: How do I adopt a pet from the local shelter?
Answer: The Aurora Bay Animal Shelter at 7 Pine Street handles adoptions. Fill out an application online or in person. Home checks may be required for some animals.
---
Result 2 (Similarity: 0.4974):
Question: How can I volunteer in community events?
Answer: You can sign up through the Community Center’s volunteer portal or attend volunteer meetings announced on the town’s website and local bulletin boards.
---
Result 3 (Similarity: 0.4980):
Question: How do I request a building permit?
Answer: Building permit applications can be obtained at the Town Hall’s Planning & Development Office or on the official website. A site inspection is required before approval.

---

Your question: quit
Thank you for using the Aurora Bay FAQ Chatbot. Goodbye!
